<a href="https://colab.research.google.com/github/vamsidh/Vamsi-s/blob/main/image_process_TEAM_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from tensorflow.keras.preprocessing import image

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, Model, models, layers
from tensorflow.keras.optimizers import RMSprop  
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow


In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/data agumentation/train'
validate_dir = '/content/drive/MyDrive/Colab Notebooks/data agumentation/validate'


In [ ]:
import pathlib
train_dir=pathlib.Path(train_dir)
validate_dir=pathlib.Path(validate_dir)

In [ ]:
x=list(train_dir.glob('*/*.jpg'))
y=list(validate_dir.glob('*/*.jpg'))

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255
 )
train_set=train_datagen.flow_from_directory(train_dir,
                                            target_size=(150,150),
                                            batch_size=8,
                                            class_mode='binary')

Found 145 images belonging to 2 classes.


In [ ]:
validate_datagen = ImageDataGenerator(rescale=1./255
)
validate_set=validate_datagen.flow_from_directory(validate_dir,
                                            target_size=(150,150),
                                            batch_size=8,
                                            class_mode='binary')
                                      

Found 37 images belonging to 2 classes.


In [ ]:
train_set.class_indices



{'not pvc': 0, 'pvc': 1}

In [ ]:
validate_set.class_indices


{'not pvc': 0, 'pvc': 1}

In [ ]:
#data agumentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(rescale=1./255,
        vertical_flip=True,
        zoom_range=0.2,
        rotation_range=45,
        horizontal_flip=True)
for image in x:
  u=load_img(image)  # this is a PIL image
  z = img_to_array(u)  # this is a Numpy array with shape (3, 150, 150)
  z = z.reshape((1,) + z.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images

  i = 0
  for batch in datagen.flow(z, batch_size=1,
                          save_to_dir='/content/drive/MyDrive/Colab Notebooks/data agumentation/train/pvc', save_prefix='pvc', save_format='jpg'):
    i += 1
    if i > 3:
        break  # otherwise the generator would loop indefinitely

In [ ]:
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',input_shape =(150,150,3)),
                            tf.keras.layers.MaxPool2D(2,2),
                            #
                            tf.keras.layers.Conv2D(32,(3,3),activation= 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            #
                            tf.keras.layers.Conv2D(64,(3,3),activation= 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            ##
                            tf.keras.layers.Flatten(),
                            ##
                            tf.keras.layers.Dense(512,activation='relu'),
                            ##
                            tf.keras.layers.Dense(1,activation='sigmoid')
]
)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer= RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model_fit= model.fit(train_set,
                     steps_per_epoch = 4,
                     epochs=30,
                     validation_data= validate_set)

Epoch 1/30
4/4 [==============================] - 10s 3s/step - loss: 1.1322 - accuracy: 0.5312 - val_loss: 1.0615 - val_accuracy: 0.4324
Epoch 2/30
4/4 [==============================] - 10s 3s/step - loss: 1.0630 - accuracy: 0.4375 - val_loss: 0.6402 - val_accuracy: 0.5676
Epoch 3/30
4/4 [==============================] - 10s 3s/step - loss: 0.6348 - accuracy: 0.6562 - val_loss: 0.5893 - val_accuracy: 0.5946
Epoch 4/30
4/4 [==============================] - 9s 3s/step - loss: 0.7711 - accuracy: 0.4800 - val_loss: 0.7324 - val_accuracy: 0.4595
Epoch 5/30
4/4 [==============================] - 10s 3s/step - loss: 0.6937 - accuracy: 0.5625 - val_loss: 0.6034 - val_accuracy: 0.6216
Epoch 6/30
4/4 [==============================] - 10s 3s/step - loss: 0.6605 - accuracy: 0.6562 - val_loss: 0.5674 - val_accuracy: 0.5946
Epoch 7/30
4/4 [==============================] - 10s 3s/step - loss: 0.5202 - accuracy: 0.6875 - val_loss: 0.5614 - val_accuracy: 0.6216
Epoch 8/30
4/4 [===================

In [ ]:
test_path='/content/drive/MyDrive/Colab Notebooks/data agumentation/test'
for i in os.listdir(test_path):
  
  img=image.load_img(test_path+ '//'+ i,target_size=(150,150))
  plt.imshow(img)
  plt.show()
  X=image.img_to_array(img)
  X= np.expand_dims(X,axis=0)
  images=np.vstack([X])

  val=model.predict(images)
  if val==0:
    print("PVC Pipes not detected")
  else:
    print("PVC pipe detected")